# Fine-Tuning for Porous, Lightweight Structures

This notebook fine-tunes the trained v3.1 MODEL C to produce more porous, lightweight architectural structures.

## Changes from Base Model
1. Increased thickness penalty (30 → 50)
2. Increased sparsity penalty (30 → 45)
3. Added new PorosityLoss
4. Reduced max_thickness (2 → 1)
5. Lower fill ratio target

In [ ]:
import os
import sys
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path

# Add project root to path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root / 'deploy'))

from model_utils import (
    UrbanPavilionNCA, 
    UrbanSceneGenerator, 
    compute_corridor_target_v31,
    LocalLegalityLoss
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

## 1. Load Pretrained Model

In [ ]:
# Paths
BASE_MODEL_DIR = Path('sel/MODEL C - Copy - no change')
CONFIG_PATH = BASE_MODEL_DIR / 'config_step_b.json'
CHECKPOINT_PATH = BASE_MODEL_DIR / 'v31_fixed_geometry.pth'

# Output directory for fine-tuned model
OUTPUT_DIR = Path('finetuned_porous')
OUTPUT_DIR.mkdir(exist_ok=True)

# Load config
with open(CONFIG_PATH, 'r') as f:
    CONFIG = json.load(f)

# Fine-tuning modifications
CONFIG['max_thickness'] = 1  # Reduced from 2
CONFIG['fill_ratio_min'] = 0.03  # Reduced from 0.05
CONFIG['fill_ratio_max'] = 0.10  # Reduced from 0.15
CONFIG['lr_finetune'] = 0.0005  # Half of original

print('Config loaded with fine-tuning modifications:')
print(f"  max_thickness: {CONFIG['max_thickness']} (was 2)")
print(f"  fill_ratio: {CONFIG.get('fill_ratio_min', 0.03)}-{CONFIG.get('fill_ratio_max', 0.10)}")
print(f"  lr_finetune: {CONFIG['lr_finetune']}")

In [ ]:
# Load pretrained model
model = UrbanPavilionNCA(CONFIG).to(device)

checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)
if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Loaded checkpoint from epoch {checkpoint.get('epoch', 'unknown')}")
else:
    model.load_state_dict(checkpoint)
    print('Loaded checkpoint (raw state dict)')

print(f'Model parameters: {sum(p.numel() for p in model.parameters()):,}')

## 2. Define Loss Functions

We import existing losses and add the new PorosityLoss.

In [ ]:
# ============ EXISTING LOSSES ============

class CorridorCoverageLoss(nn.Module):
    """Penalize uncovered corridor voxels."""
    def __init__(self, config: dict):
        super().__init__()
        self.config = config

    def forward(self, structure: torch.Tensor, corridor_target: torch.Tensor) -> torch.Tensor:
        corridor_mask = corridor_target > 0.5
        corridor_volume = corridor_mask.float().sum() + 1e-8
        covered = (structure * corridor_mask.float()).sum()
        coverage_ratio = covered / corridor_volume
        return F.relu(0.7 - coverage_ratio)  # Target 70% coverage


class CorridorSpillLoss(nn.Module):
    """Penalize structure outside corridor."""
    def __init__(self, config: dict):
        super().__init__()
        self.config = config

    def forward(self, structure: torch.Tensor, corridor_target: torch.Tensor, 
                legality_field: torch.Tensor) -> torch.Tensor:
        outside_corridor = (corridor_target < 0.5).float()
        legal_outside = outside_corridor * legality_field
        spill = (structure * legal_outside).sum()
        total_structure = structure.sum() + 1e-8
        return spill / total_structure


class GroundOpennessLoss(nn.Module):
    """Keep ground level open except at anchors."""
    def __init__(self, config: dict):
        super().__init__()
        self.config = config
        self.street_levels = config['street_levels']

    def forward(self, structure: torch.Tensor, corridor_target: torch.Tensor,
                legality_field: torch.Tensor) -> torch.Tensor:
        ground = structure[:, :self.street_levels]
        corridor_ground = corridor_target[:, :self.street_levels]
        outside_corridor_ground = (corridor_ground < 0.5).float()
        unwanted = ground * outside_corridor_ground
        return unwanted.mean()


class ThicknessLoss(nn.Module):
    """Penalize thick/bulky structures."""
    def __init__(self, max_thickness: int = 1):
        super().__init__()
        self.max_thickness = max_thickness

    def forward(self, structure: torch.Tensor) -> torch.Tensor:
        binary = (structure > 0.5).float()
        eroded = binary.clone()
        for _ in range(self.max_thickness):
            eroded = -F.max_pool3d(-eroded.unsqueeze(0).unsqueeze(0), 3, 1, 1).squeeze()
        core = eroded
        core_ratio = core.sum() / (binary.sum() + 1e-8)
        return core_ratio


class SparsityLossV31(nn.Module):
    """Squared sparsity penalty."""
    def __init__(self, target_ratio: float = 0.08, squared: bool = True):
        super().__init__()
        self.target = target_ratio
        self.squared = squared

    def forward(self, structure: torch.Tensor, available: torch.Tensor) -> torch.Tensor:
        available_vol = available.sum() + 1e-8
        filled = structure.sum()
        ratio = filled / available_vol
        excess = F.relu(ratio - self.target)
        return excess ** 2 if self.squared else excess


class DensityPenalty(nn.Module):
    """Overall density penalty."""
    def __init__(self, target: float = 0.05):
        super().__init__()
        self.target = target

    def forward(self, structure: torch.Tensor) -> torch.Tensor:
        density = structure.mean()
        return F.relu(density - self.target)


class TotalVariation3D(nn.Module):
    """Smoothness regularization."""
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        dz = torch.abs(x[:, 1:, :, :] - x[:, :-1, :, :]).mean()
        dy = torch.abs(x[:, :, 1:, :] - x[:, :, :-1, :]).mean()
        dx = torch.abs(x[:, :, :, 1:] - x[:, :, :, :-1]).mean()
        return (dz + dy + dx) / 3


class AccessConnectivityLoss(nn.Module):
    """Ensure access points are connected."""
    def __init__(self, config: dict):
        super().__init__()
        self.config = config

    def forward(self, state: torch.Tensor) -> torch.Tensor:
        cfg = self.config
        structure = state[:, cfg['ch_structure']]
        access = state[:, cfg['ch_access']]
        dilated = F.max_pool3d(structure.unsqueeze(1), 3, 1, 1).squeeze(1)
        connected = (access * dilated).sum()
        total_access = access.sum() + 1e-8
        return 1.0 - (connected / total_access)


class LoadPathLoss(nn.Module):
    """Ensure vertical load paths."""
    def __init__(self, config: dict):
        super().__init__()
        self.config = config

    def forward(self, state: torch.Tensor) -> torch.Tensor:
        cfg = self.config
        structure = state[:, cfg['ch_structure']]
        B, D, H, W = structure.shape
        ground_support = structure[:, 0:1].clone()
        supported = ground_support
        for z in range(1, D):
            below = F.max_pool3d(supported.unsqueeze(1), (1, 3, 3), 1, (0, 1, 1)).squeeze(1)
            supported = torch.max(supported, torch.min(structure[:, z:z+1], below[:, -1:]))
        unsupported = F.relu(structure - supported.expand_as(structure))
        return unsupported.mean()


class CantileverLoss(nn.Module):
    """Limit overhangs."""
    def forward(self, structure: torch.Tensor) -> torch.Tensor:
        below = F.pad(structure[:, :-1], (0, 0, 0, 0, 1, 0))
        unsupported = F.relu(structure - below)
        return unsupported.mean()


class FacadeContactLoss(nn.Module):
    """Limit contact with existing buildings."""
    def __init__(self, config: dict, max_contact_ratio: float = 0.15):
        super().__init__()
        self.config = config
        self.max_ratio = max_contact_ratio

    def forward(self, state: torch.Tensor) -> torch.Tensor:
        cfg = self.config
        structure = state[:, cfg['ch_structure']]
        existing = state[:, cfg['ch_existing']]
        dilated_existing = F.max_pool3d(existing.unsqueeze(1), 3, 1, 1).squeeze(1)
        facade_zone = dilated_existing * (1 - existing)
        contact = (structure * facade_zone).sum()
        total_struct = structure.sum() + 1e-8
        ratio = contact / total_struct
        return F.relu(ratio - self.max_ratio)


print('Existing losses defined')

In [ ]:
# ============ NEW POROSITY LOSS ============

class PorosityLoss(nn.Module):
    """
    Encourage internal voids within the structure.
    Penalizes structures where interior voxels are filled.
    
    The idea: erode the structure, what remains is "interior".
    We want some of that interior to be void (not filled).
    """
    def __init__(self, target_porosity: float = 0.25, erosion_steps: int = 1):
        super().__init__()
        self.target_porosity = target_porosity
        self.erosion_steps = erosion_steps

    def forward(self, structure: torch.Tensor, corridor: torch.Tensor) -> torch.Tensor:
        # Binary structure
        binary = (structure > 0.5).float()
        
        # Erode to find interior
        eroded = binary.clone()
        for _ in range(self.erosion_steps):
            eroded = -F.max_pool3d(
                -eroded.unsqueeze(0).unsqueeze(0), 3, 1, 1
            ).squeeze(0).squeeze(0)
        
        # Interior mask (eroded structure = definitely inside)
        interior_mask = eroded > 0.5
        
        if interior_mask.sum() < 10:  # Not enough interior to measure
            return torch.tensor(0.0, device=structure.device)
        
        # Compute how much of the interior is filled
        interior_filled = (structure * interior_mask.float()).sum()
        interior_total = interior_mask.float().sum()
        
        # Porosity = 1 - fill_ratio in interior
        fill_ratio = interior_filled / (interior_total + 1e-8)
        porosity = 1.0 - fill_ratio
        
        # Penalize if porosity below target
        return F.relu(self.target_porosity - porosity)


class SurfaceAreaLoss(nn.Module):
    """
    Encourage higher surface-to-volume ratio (more articulated forms).
    Higher surface area relative to volume = more porous/lattice-like.
    """
    def __init__(self, target_ratio: float = 3.0):
        super().__init__()
        self.target_ratio = target_ratio

    def forward(self, structure: torch.Tensor) -> torch.Tensor:
        binary = (structure > 0.5).float()
        volume = binary.sum() + 1e-8
        
        # Compute surface by counting face exposures
        # For each direction, count transitions from filled to empty
        padded = F.pad(binary.unsqueeze(0).unsqueeze(0), (1,1,1,1,1,1), value=0).squeeze()
        
        surface = 0
        # Z direction
        surface += torch.abs(padded[1:, :, :] - padded[:-1, :, :]).sum()
        # Y direction  
        surface += torch.abs(padded[:, 1:, :] - padded[:, :-1, :]).sum()
        # X direction
        surface += torch.abs(padded[:, :, 1:] - padded[:, :, :-1]).sum()
        
        ratio = surface / volume
        
        # Penalize if ratio below target (we want HIGH surface area)
        return F.relu(self.target_ratio - ratio)


print('New porosity losses defined')
print(f'  PorosityLoss: target_porosity=0.25')
print(f'  SurfaceAreaLoss: target_ratio=3.0')

## 3. Fine-Tuning Trainer

In [ ]:
class PorosityFineTuner:
    """
    Fine-tuning trainer with modified weights and new porosity losses.
    """
    def __init__(self, model: nn.Module, config: dict):
        self.model = model
        self.config = config
        cfg = config
        
        max_thickness = cfg.get('max_thickness', 1)
        max_facade = cfg.get('max_facade_contact', 0.15)
        
        # Loss functions
        self.legality_loss = LocalLegalityLoss(cfg)
        self.coverage_loss = CorridorCoverageLoss(cfg)
        self.spill_loss = CorridorSpillLoss(cfg)
        self.ground_loss = GroundOpennessLoss(cfg)
        self.thickness_loss = ThicknessLoss(max_thickness)
        self.sparsity_loss = SparsityLossV31(target_ratio=0.06, squared=True)
        self.facade_loss = FacadeContactLoss(cfg, max_facade)
        self.access_conn_loss = AccessConnectivityLoss(cfg)
        self.loadpath_loss = LoadPathLoss(cfg)
        self.cantilever_loss = CantileverLoss()
        self.density_loss = DensityPenalty(target=0.04)
        self.tv_loss = TotalVariation3D()
        
        # NEW: Porosity losses
        self.porosity_loss = PorosityLoss(target_porosity=0.25)
        self.surface_loss = SurfaceAreaLoss(target_ratio=2.5)
        
        # MODIFIED WEIGHTS for porosity fine-tuning
        self.weights = {
            'legality': 30.0,
            'coverage': 20.0,       # Slightly reduced
            'spill': 25.0,
            'ground': 35.0,
            'thickness': 50.0,      # INCREASED (was 30)
            'sparsity': 45.0,       # INCREASED (was 30)
            'facade': 10.0,
            'access_conn': 15.0,
            'loadpath': 10.0,       # Slightly increased for stability
            'cantilever': 5.0,
            'density': 8.0,         # INCREASED (was 3)
            'tv': 0.5,              # REDUCED (was 1) - allow more variation
            'porosity': 20.0,       # NEW
            'surface': 10.0,        # NEW
        }
        
        self.optimizer = torch.optim.Adam(
            model.parameters(), 
            lr=cfg.get('lr_finetune', 0.0005)
        )
        self.scene_gen = UrbanSceneGenerator(cfg)
        self.history = []
        
    def train_epoch(self, epoch: int) -> dict:
        self.model.train()
        cfg = self.config
        batch_size = cfg.get('batch_size', 4)
        steps_range = (40, 60)
        
        # Generate batch of scenes
        seeds = []
        corridors = []
        
        for _ in range(batch_size):
            params = self._random_scene_params()
            seed, info = self.scene_gen.generate(params, device=device)
            corridor = compute_corridor_target_v31(
                seed, cfg,
                corridor_width=cfg.get('corridor_width', 1),
                vertical_envelope=cfg.get('vertical_envelope', 1)
            )
            seeds.append(seed)
            corridors.append(corridor)
        
        seeds = torch.cat(seeds, dim=0)
        corridor_target = torch.cat(corridors, dim=0)
        
        # Apply corridor seeding
        seed_scale = cfg.get('corridor_seed_scale', 0.15)
        if seed_scale > 0:
            seeds[:, cfg['ch_structure']] = torch.clamp(
                seeds[:, cfg['ch_structure']] + seed_scale * corridor_target, 0, 1
            )
        
        # Forward pass
        self.optimizer.zero_grad()
        steps = np.random.randint(*steps_range)
        final = self.model(seeds, steps=steps)
        
        structure = final[:, cfg['ch_structure']]
        existing = final[:, cfg['ch_existing']]
        available = 1.0 - existing
        legality_field = self.legality_loss.compute_legality_field(final)
        
        # Compute all losses
        L_legality = self.legality_loss(final)
        L_coverage = self.coverage_loss(structure, corridor_target)
        L_spill = self.spill_loss(structure, corridor_target, legality_field)
        L_ground = self.ground_loss(structure, corridor_target, legality_field)
        L_thickness = self.thickness_loss(structure)
        L_sparsity = self.sparsity_loss(structure, available)
        L_facade = self.facade_loss(final)
        L_access = self.access_conn_loss(final)
        L_loadpath = self.loadpath_loss(final)
        L_cant = self.cantilever_loss(structure)
        L_density = self.density_loss(structure)
        L_tv = self.tv_loss(structure)
        
        # NEW: Porosity losses
        L_porosity = self.porosity_loss(structure, corridor_target)
        L_surface = self.surface_loss(structure)
        
        # Total loss
        total_loss = (
            self.weights['legality'] * L_legality +
            self.weights['coverage'] * L_coverage +
            self.weights['spill'] * L_spill +
            self.weights['ground'] * L_ground +
            self.weights['thickness'] * L_thickness +
            self.weights['sparsity'] * L_sparsity +
            self.weights['facade'] * L_facade +
            self.weights['access_conn'] * L_access +
            self.weights['loadpath'] * L_loadpath +
            self.weights['cantilever'] * L_cant +
            self.weights['density'] * L_density +
            self.weights['tv'] * L_tv +
            self.weights['porosity'] * L_porosity +
            self.weights['surface'] * L_surface
        )
        
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), cfg.get('grad_clip', 1.0))
        self.optimizer.step()
        
        metrics = {
            'epoch': epoch,
            'total_loss': total_loss.item(),
            'L_coverage': L_coverage.item(),
            'L_spill': L_spill.item(),
            'L_thickness': L_thickness.item(),
            'L_sparsity': L_sparsity.item(),
            'L_porosity': L_porosity.item(),
            'L_surface': L_surface.item(),
            'fill_ratio': structure.mean().item(),
        }
        self.history.append(metrics)
        return metrics
    
    def _random_scene_params(self) -> dict:
        """Generate random scene parameters."""
        G = self.config['grid_size']
        gap_width = np.random.randint(8, 16)
        gap_center = G // 2
        
        left_x = (0, gap_center - gap_width // 2)
        right_x = (gap_center + gap_width // 2, G)
        
        buildings = [
            {
                'x': list(left_x),
                'y': [0, np.random.randint(20, 28)],
                'z': [0, np.random.randint(10, 18)],
                'side': 'left',
                'gap_facing_x': left_x[1]
            },
            {
                'x': list(right_x),
                'y': [0, np.random.randint(16, 24)],
                'z': [0, np.random.randint(10, 18)],
                'side': 'right',
                'gap_facing_x': right_x[0]
            }
        ]
        
        access_points = [
            {'x': gap_center, 'y': np.random.randint(18, 26), 'z': 0, 'type': 'ground'},
        ]
        
        # Sometimes add elevated access
        if np.random.random() > 0.3:
            access_points.append({
                'x': np.random.choice([left_x[1], right_x[0]]),
                'y': np.random.randint(2, 8),
                'z': np.random.randint(6, 12),
                'type': 'elevated'
            })
        
        return {'buildings': buildings, 'access_points': access_points}
    
    def evaluate(self, n_samples: int = 8) -> dict:
        """Evaluate on random samples."""
        self.model.eval()
        cfg = self.config
        results = []
        
        with torch.no_grad():
            for _ in range(n_samples):
                params = self._random_scene_params()
                seed, _ = self.scene_gen.generate(params, device=device)
                corridor = compute_corridor_target_v31(seed, cfg)
                
                seed_scale = cfg.get('corridor_seed_scale', 0.15)
                if seed_scale > 0:
                    seed[:, cfg['ch_structure']] += seed_scale * corridor
                
                grown = self.model(seed, steps=50)
                structure = grown[:, cfg['ch_structure']]
                
                # Compute metrics
                corridor_mask = corridor > 0.5
                coverage = (structure * corridor_mask.float()).sum() / (corridor_mask.sum() + 1e-8)
                
                outside = (corridor < 0.5).float()
                spill = (structure * outside).sum() / (structure.sum() + 1e-8)
                
                fill_ratio = structure.mean().item()
                
                results.append({
                    'coverage': coverage.item(),
                    'spill': spill.item(),
                    'fill_ratio': fill_ratio,
                })
        
        return {
            'avg_coverage': np.mean([r['coverage'] for r in results]),
            'avg_spill': np.mean([r['spill'] for r in results]),
            'avg_fill_ratio': np.mean([r['fill_ratio'] for r in results]),
        }
    
    def save_checkpoint(self, path: str, epoch: int):
        """Save checkpoint."""
        torch.save({
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'config': self.config,
            'weights': self.weights,
            'history': self.history,
        }, path)
        print(f'Saved checkpoint to {path}')


print('PorosityFineTuner defined')

## 4. Run Fine-Tuning

In [ ]:
# Create trainer
trainer = PorosityFineTuner(model, CONFIG)

print('Fine-tuning weights:')
for name, weight in trainer.weights.items():
    print(f'  {name}: {weight}')

In [ ]:
# Fine-tuning parameters
N_EPOCHS = 150
EVAL_EVERY = 10
SAVE_EVERY = 25

print(f'Starting fine-tuning for {N_EPOCHS} epochs...')
print(f'  Eval every {EVAL_EVERY} epochs')
print(f'  Save every {SAVE_EVERY} epochs')
print()

In [ ]:
# Training loop
best_fill_ratio = 1.0

for epoch in range(1, N_EPOCHS + 1):
    metrics = trainer.train_epoch(epoch)
    
    if epoch % 5 == 0:
        print(
            f"Epoch {epoch:4d} | Loss: {metrics['total_loss']:.2f} | "
            f"Fill: {metrics['fill_ratio']*100:.1f}% | "
            f"Poros: {metrics['L_porosity']:.3f} | "
            f"Thick: {metrics['L_thickness']:.3f}"
        )
    
    if epoch % EVAL_EVERY == 0:
        eval_results = trainer.evaluate(n_samples=8)
        print(f"  [EVAL] Coverage: {eval_results['avg_coverage']*100:.1f}% | "
              f"Spill: {eval_results['avg_spill']*100:.1f}% | "
              f"Fill: {eval_results['avg_fill_ratio']*100:.1f}%")
        
        # Save best by fill ratio (lower is better for porosity)
        if eval_results['avg_fill_ratio'] < best_fill_ratio and eval_results['avg_coverage'] > 0.5:
            best_fill_ratio = eval_results['avg_fill_ratio']
            trainer.save_checkpoint(OUTPUT_DIR / 'best_porous.pth', epoch)
    
    if epoch % SAVE_EVERY == 0:
        trainer.save_checkpoint(OUTPUT_DIR / f'checkpoint_epoch_{epoch}.pth', epoch)

# Final save
trainer.save_checkpoint(OUTPUT_DIR / 'final_porous.pth', N_EPOCHS)
print('\nFine-tuning complete!')

## 5. Visualize Results

In [ ]:
# Plot training history
history = trainer.history
epochs = [h['epoch'] for h in history]

fig, axes = plt.subplots(2, 3, figsize=(14, 8))

axes[0, 0].plot(epochs, [h['total_loss'] for h in history])
axes[0, 0].set_title('Total Loss')
axes[0, 0].set_xlabel('Epoch')

axes[0, 1].plot(epochs, [h['L_porosity'] for h in history], 'g')
axes[0, 1].set_title('Porosity Loss')
axes[0, 1].set_xlabel('Epoch')

axes[0, 2].plot(epochs, [h['L_thickness'] for h in history], 'purple')
axes[0, 2].set_title('Thickness Loss')
axes[0, 2].set_xlabel('Epoch')

axes[1, 0].plot(epochs, [h['fill_ratio'] * 100 for h in history], 'orange')
axes[1, 0].set_title('Fill Ratio %')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].axhline(y=10, color='r', linestyle='--', alpha=0.5, label='Target max')

axes[1, 1].plot(epochs, [h['L_coverage'] for h in history], 'b')
axes[1, 1].set_title('Coverage Loss')
axes[1, 1].set_xlabel('Epoch')

axes[1, 2].plot(epochs, [h['L_sparsity'] for h in history], 'r')
axes[1, 2].set_title('Sparsity Loss')
axes[1, 2].set_xlabel('Epoch')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'training_curves.png', dpi=150)
plt.show()

In [ ]:
# Compare original vs fine-tuned
def visualize_comparison(original_model, finetuned_model, config, seed=42):
    """Generate and visualize comparison."""
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    scene_gen = UrbanSceneGenerator(config)
    params = {
        'buildings': [
            {'x': [1, 9], 'y': [0, 27], 'z': [0, 14], 'side': 'left', 'gap_facing_x': 9},
            {'x': [23, 32], 'y': [0, 19], 'z': [0, 14], 'side': 'right', 'gap_facing_x': 23}
        ],
        'access_points': [
            {'x': 12, 'y': 23, 'z': 0, 'type': 'ground'},
            {'x': 9, 'y': 1, 'z': 11, 'type': 'elevated'}
        ]
    }
    
    seed_state, _ = scene_gen.generate(params, device=device)
    corridor = compute_corridor_target_v31(seed_state, config)
    
    seed_scale = config.get('corridor_seed_scale', 0.15)
    if seed_scale > 0:
        seed_state[:, config['ch_structure']] += seed_scale * corridor
    
    with torch.no_grad():
        original_model.eval()
        finetuned_model.eval()
        
        orig_result = original_model(seed_state.clone(), steps=50)
        fine_result = finetuned_model(seed_state.clone(), steps=50)
    
    orig_struct = (orig_result[0, config['ch_structure']] > 0.5).cpu().numpy()
    fine_struct = (fine_result[0, config['ch_structure']] > 0.5).cpu().numpy()
    
    print(f'Original fill ratio: {orig_struct.mean()*100:.1f}%')
    print(f'Fine-tuned fill ratio: {fine_struct.mean()*100:.1f}%')
    print(f'Reduction: {(1 - fine_struct.mean()/orig_struct.mean())*100:.1f}%')
    
    return orig_struct, fine_struct

# Load original model for comparison
original_model = UrbanPavilionNCA(CONFIG).to(device)
orig_checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)
if isinstance(orig_checkpoint, dict) and 'model_state_dict' in orig_checkpoint:
    original_model.load_state_dict(orig_checkpoint['model_state_dict'])
else:
    original_model.load_state_dict(orig_checkpoint)

orig_struct, fine_struct = visualize_comparison(original_model, model, CONFIG)

## 6. Export for Deployment

In [ ]:
# Copy best model to deploy folder
import shutil

deploy_model_dir = project_root / 'deploy' / 'models'
deploy_model_dir.mkdir(exist_ok=True)

best_checkpoint = OUTPUT_DIR / 'best_porous.pth'
if best_checkpoint.exists():
    shutil.copy(best_checkpoint, deploy_model_dir / 'v31_porous.pth')
    print(f'Copied best model to {deploy_model_dir / "v31_porous.pth"}')
else:
    final_checkpoint = OUTPUT_DIR / 'final_porous.pth'
    shutil.copy(final_checkpoint, deploy_model_dir / 'v31_porous.pth')
    print(f'Copied final model to {deploy_model_dir / "v31_porous.pth"}')

# Also save config
with open(deploy_model_dir / 'config_porous.json', 'w') as f:
    json.dump(CONFIG, f, indent=2)
print(f'Saved config to {deploy_model_dir / "config_porous.json"}')

## Summary

Fine-tuning complete! The model has been trained with:

1. **Increased thickness penalty** (30 → 50)
2. **Increased sparsity penalty** (30 → 45)
3. **New PorosityLoss** targeting 25% internal voids
4. **New SurfaceAreaLoss** encouraging articulated forms
5. **Reduced max_thickness** (2 → 1)
6. **Lower fill ratio target** (5-15% → 3-10%)

The fine-tuned model should produce more porous, lattice-like structures while maintaining corridor coverage and structural validity.